# checks

> Phase 1: Prerequisites Validation  
> Verify system readiness before any destructive operations.

In [ ]:
#| default_exp checks

Before creating repositories, configuring environments, or running builds, we validate that all required tools exist and are properly configured. This fail-fast approach prevents partial initialization states that leave projects in limbo.

The checks fall into three categories: **command availability** (is the tool installed?), **authentication state** (can we talk to GitHub?), and **user metadata** (who is creating this project?).

In [ ]:
#| export
import sys
import subprocess
from pj.core import check_cmd, run_cmd, get_git_config, hr

## Prerequisite Validation

The validation orchestrator: check every required tool, verify GitHub authentication, and gather user information for project metadata. Returns a dictionary of user info if all checks pass; exits immediately if anything fails.

In [ ]:
#| export
def check_prereqs():
    """
    Validate all prerequisites and gather system info.
    Returns dict with user info or exits on failure.
    """
    print(hr * 60)
    print("PHASE 1: CHECKS")
    
    # Check required tools
    checks = [
        ("gh", "sudo apt install gh  # See https://github.com/cli/cli/blob/trunk/docs/install_linux.md"),
        ("uv", "curl -LsSf https://astral.sh/uv/install.sh | sh"),
        ("direnv", "sudo apt install direnv  # Then add: eval \"$(direnv hook bash)\" to ~/.bashrc"),
        ("nbdev_new", "uv tool install nbdev  # or: pipx install nbdev"),
        ("git", "sudo apt install git"),
    ]
    
    all_ok = True
    for cmd, hint in checks:
        if not check_cmd(cmd, hint): all_ok = False
    
    if not all_ok: sys.exit(1)
    
    # Check GitHub auth
    try:
        result = subprocess.run(["gh", "auth", "status"], capture_output=True, text=True)
        if result.returncode != 0:
            print("❌ Error: GitHub CLI is not authenticated")
            print("   Run: gh auth login")
            sys.exit(1)
    except Exception:
        print("❌ Error: Cannot verify GitHub authentication")
        sys.exit(1)
    
    print("✅ All prerequisites satisfied")
    
    # Gather user info
    git_user = get_git_config("user.name") or "Your Name"
    git_email = get_git_config("user.email") or "you@example.com"
    
    # Get GitHub username
    gh_user_result = subprocess.run(["gh", "api", "user", "--jq", ".login"], capture_output=True, text=True, check=False)
    gh_username = gh_user_result.stdout.strip() if gh_user_result.returncode == 0 else "username"
    
    return {
        'git_user': git_user,
        'git_email': git_email,
        'gh_username': gh_username
    }

### Design Decisions

**Why check commands individually?** We want to report *all* missing tools at once, not fail on the first missing one. The user sees the complete list of what needs installing, rather than playing whack-a-mole across multiple runs.

**Why validate `nbdev_new` specifically?** There's no `nbdev` command—it's a collection of subcommands like `nbdev_new`, `nbdev_export`, `nbdev_prepare`. Checking for a non-existent parent command would pass even when nbdev isn't installed.

**Why separate GitHub auth check?** Authentication is stateful and can expire. We verify it explicitly with `gh auth status` rather than letting a later `gh repo create` fail cryptically.

**Why gather user info here?** Git and GitHub metadata (author name, email, username) are needed for project scaffolding. Collecting them during checks means we fail early if git isn't configured, and we have the data ready when needed later.

**Default values for missing config:** If `user.name` or `user.email` aren't set, we provide placeholder strings. `nbdev_new` will use these values, and the user can fix them later in `settings.ini`. Better than blocking on missing config.

The function returns a dictionary rather than printing values because the caller (`init_nbdev`) needs to pass this data to subsequent setup steps. Returning structured data keeps concerns separated: checks validate, initialization consumes.